In [34]:
import sys
import json
from collections import defaultdict
from url_normalize import url_normalize
from urlparse import urlparse
import numpy as np

def norm(url):
    url_n = url_normalize(url)
    parsed = urlparse(url_n)
    scheme = "%s://" % parsed.scheme
    res = parsed.geturl().replace(scheme, '', 1)
    host = parsed.netloc
    if res.startswith('www.'):
        res = res.replace('www.', '')
        host = host.replace('www.', '')
    return res, host

count = defaultdict(int)

data = json.load(open('/home/akulov/durov/data/out_link_ru.json'))

url_set = set()
host_set = set()

host_to_host = defaultdict(lambda: defaultdict(int))

for row in data:
    url = row['url']
    url_n, url_host = norm(url)
    url_set.add(url_n)
    host_set.add(url_host)

In [35]:
for row in data:
    url = row['url']
    url_n, url_host = norm(url)
    for link in row['out_links']:
        link_n, link_host = norm(link)

        #if url != url_n or link != link_n:
        #    print url, link
        #    print url_n, link_n
        #    print url_host, link_host
        #    print ""
        if url_host != link_host and link_n in url_set:
            count[link] += 1
        if url_host != link_host and link_host in host_set:
            host_to_host[url_host][link_host] += 1
            #print url_host, link_host

size = 0
h2i = {}
i2h = {}
for host in host_to_host:
    h2i[host] = size
    i2h[size] = host
    size += 1

In [36]:
matrix = np.zeros((size,size))

for host in host_to_host:
    for link_host in host_to_host[host]:
        if link_host in h2i:
            matrix[h2i[host]][h2i[link_host]] = host_to_host[host][link_host]



In [37]:
for i in xrange(0,size):
    if np.sum(matrix[i]) == 0:
        matrix[i] = np.ones(size)

In [38]:
matrix = matrix / np.sum(matrix, axis=1, keepdims=True)

In [39]:
x = np.ones((1,size))
x = x / np.sum(x, axis=1, keepdims=True)

In [40]:
y = np.dot(x, np.linalg.matrix_power(matrix,1))

In [41]:
rating = []
for i in xrange(0,size):
    rating.append((i, y[0][i], i2h[i]))
rating = sorted(rating, key=lambda x:-x[1])

In [42]:
for i, pr, host in rating:
    print pr, host

0.09492476134065407 ria.ru
0.07594162670589077 kommersant.ru
0.04667236025374613 iz.ru
0.04515468915567602 rbc.ru
0.031045270533541225 interfax.ru
0.026030102020032368 rg.ru
0.019572634571107164 lenta.ru
0.016129881983984323 russian.rt.com
0.015727363690930373 meduza.io
0.014247850171894894 pravda.com.ua
0.013158560715952032 nv.ua
0.011701790152200454 novayagazeta.ru
0.0113407850853374 mskagency.ru
0.010712943538283304 fontanka.ru
0.010423918368767457 sports.ru
0.010030613957596121 interfax.com.ua
0.009890780652263008 rsport.ria.ru
0.009781439231068712 tjournal.ru
0.008863880447469734 nation-news.ru
0.008207215744231855 znak.com
0.007974019825601211 ura.news
0.007326147357044599 strana.ua
0.00709869675166683 korrespondent.net
0.006854788113072688 24tv.ua
0.0066733606637448 dumskaya.net
0.006604640056000125 360tv.ru
0.006471616345626383 radiosputnik.ria.ru
0.0063433015111712295 obozrevatel.com
0.006064147199016042 newizv.ru
0.006005511583927032 life.ru
0.00593625851610326 interfax-russi

In [43]:
out = open('/home/akulov/durov/data/ru_rating.txt', 'w')
for i, pr, host in rating:
    out.write('%f\t%s\n' % (pr, host))